In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [1]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE


## Notebook overview
- Get insight into tokenizer, tokens and doc lengths.
- Test different text truncation thresholds on the baseline.

#### Text truncation -- overview in tokenizer/doc lengths
- check how many docs exceed max

- first, tokenize text using tokenizer of mistral, geitje and Llama.
- Check if mistral and geitje indeed have the same tokenizer.
- After getting the tokens, check distribution and how many exceed max_threshold.

In [3]:
import pandas as pd
df = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")

In [13]:
from transformers import AutoTokenizer

def get_tokens(model_name, df, save_to_path, text_col, new_col_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    all_texts = list(df[text_col].values)

    all_tokens = []
    all_tokens_len = []
    for txt in all_texts:
        tokens = tokenizer.tokenize(txt)
        all_tokens.append(tokens)
        all_tokens_len.append(len(tokens))

    df[new_col_name] = all_tokens
    df[f"count_{new_col_name}"] = all_tokens_len
    df.to_pickle(save_to_path)
    return df

# subdf = df.iloc[0:2]
# # display(subdf)
# get_token_length('Rijgersberg/GEITje-7B-chat-v2', subdf, f"{cf.output_path}/try_out_token_count.pkl", 'text', 'token_count_geitje')

def fraction_token(df, max_token, token_len_col):
    for col in token_len_col:
        print(f"{len(df.loc[df[col]>max_token])} out of {len(df)} ({round(len(df.loc[df[col]>max_token])/len(df)*100, 2)}%) docs exceed a token length of {max_token}")

    for col in token_len_col:
        print(df[col].describe())

    


    

/anaconda/envs/AmsterdamInContextLearning/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


##### Get token lengths of model tokenizers

In [16]:
"""GEITje""" ## not necesarry -> since tokenizer is the same as mistral
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_tokens('Rijgersberg/GEITje-7B-chat-v2', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'GEITjeTokens')

"""Mistral"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_tokens('mistralai/Mistral-7B-v0.1', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'MistralTokens')

"""Llama"""
# df = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# get_tokens('meta-llama/Llama-2-7b-hf', df, f"{cf.output_path}/txtfiles_tokenizer.pkl", 'text', 'LlamaTokens')

,label,path,id,text,tokens,token_count,clean_tokens,clean_tokens_count,num_pages,clean_text,4split,2split,MistralTokens,count_MistralTokens,LlamaTokens,count_LlamaTokens
8977,Agenda,/home/azureuser/cloudfiles/code/blobfuse/raads...,15789,X Gemeente Amsterdam J C\n% Raadscommissie voo...,"[X, Gemeente, Amsterdam, J, C, %, Raadscommiss...",801,"[Gemeente, Amsterdam, Raadscommissie, Onderwij...",432,4.0,Gemeente Amsterdam Raadscommissie Onderwijs Je...,train,train,"[▁X, ▁Geme, ente, ▁Amsterdam, ▁J, ▁C, <0x0A>, ...",1947,"[▁X, ▁Geme, ente, ▁Amsterdam, ▁J, ▁C, <0x0A>, ...",1896
19264,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,31012,x Gemeente Amsterdam R\nGemeenteraad\n% Gemeen...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",824,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",428,3.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad S...,train,train,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",1841,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",1774
2900,Besluit,/home/azureuser/cloudfiles/code/blobfuse/raads...,20680,> < Gemeente Amsterdam\nStadsdeel Centrum\nx P...,"[>, <, Gemeente, Amsterdam, Stadsdeel, Centrum...",1204,"[Gemeente, Amsterdam, Stadsdeel, Centrum, Publ...",588,4.0,Gemeente Amsterdam Stadsdeel Centrum Publicati...,train,train,"[▁>, ▁<, ▁Geme, ente, ▁Amsterdam, <0x0A>, St, ...",2606,"[▁>, ▁<, ▁Geme, ente, ▁Amsterdam, <0x0A>, St, ...",2527
18869,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,32499,x Gemeente Amsterdam R\nGemeenteraad\n% Gemeen...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",897,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",436,3.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad S...,train,train,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",1990,"[▁x, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",1917
25666,Actualiteit,/home/azureuser/cloudfiles/code/blobfuse/raads...,10589,% Gemeente Amsterdam\n% Actualiteit voor de ra...,"[%, Gemeente, Amsterdam, %, Actualiteit, voor,...",302,"[Gemeente, Amsterdam, Actualiteit, raadscommis...",155,1.0,Gemeente Amsterdam Actualiteit raadscommissie ...,train,train,"[▁%, ▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Act,...",683,"[▁%, ▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Act,...",653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13909,Termijnagenda,/home/azureuser/cloudfiles/code/blobfuse/raads...,25729,Termijnagenda raadscommissie SL 4 december 201...,"[Termijnagenda, raadscommissie, SL, 4, decembe...",729,"[Termijnagenda, raadscommissie, SL, december, ...",484,6.0,Termijnagenda raadscommissie SL december 2012 ...,dev,train,"[▁Term, ijn, ag, enda, ▁ra, ads, comm, iss, ie...",1880,"[▁Term, ijn, ag, enda, ▁ra, ads, comm, iss, ie...",1857
6388,Agenda,/home/azureuser/cloudfiles/code/blobfuse/raads...,15435,D Gemeente Amsterdam W l J\n% Raadscommissie v...,"[D, Gemeente, Amsterdam, W, l, J, %, Raadscomm...",598,"[Gemeente, Amsterdam, Raadscommissie, Werk, In...",333,3.0,Gemeente Amsterdam Raadscommissie Werk Inkomen...,dev,train,"[▁D, ▁Geme, ente, ▁Amsterdam, ▁W, ▁l, ▁J, <0x0...",1552,"[▁D, ▁Geme, ente, ▁Amsterdam, ▁W, ▁l, ▁J, <0x0...",1545
9410,Voordracht,/home/azureuser/cloudfiles/code/blobfuse/raads...,28982,VN2022-026391 N% Gemeente De raadscommissie vo...,"[VN2022-026391, N, %, Gemeente, De, raadscommi...",306,"[VN2022-026391, Gemeente, raadscommissie, Woni...",180,2.0,VN2022-026391 Gemeente raadscommissie Woningbo...,dev,train,"[▁V, N, 2, 0, 2, 2, -, 0, 2, 6, 3, 9, 1, ▁N, %...",813,"[▁V, N, 2, 0, 2, 2, -, 0, 2, 6, 3, 9, 1, ▁N, %...",806
16340,Raadsadres,/home/azureuser/cloudfiles/code/blobfuse/raads...,24699,Van: M. Bejaoui [mailto:m.bejaoui@hotmail.nl]\...,"[Van, :, M., Bejaoui, [, mailto, :, m.bejaoui,...",469,"[M., Bejaoui, mailto, m.bejaoui, hotmail.nl, V...",213,2.0,M. Bejaoui mailto m.bejaoui hotmail.nl Verzond...,dev,train,"[▁Van, :, ▁M, ., ▁Be, ja, ou, i, ▁[, mail, to,...",999,"[▁Van, :, ▁M, ., ▁Be, ja, ou, i, ▁[, mail, to,...",961


##### Analyse token length of model tokenizers

In [2]:
df_token_len = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
fraction_token(df_token_len, 4096, ['count_MistralTokens', 'count_LlamaTokens'])

NameError: name 'pd' is not defined

#### Test text truncation on baseline

In [2]:
import pandas as pd
tok = pd.read_pickle(f"{cf.output_path}/txtfiles_tokenizer.pkl")
# display(tok)

In [3]:
def add_truncation_column(df,text_col, token_col_name, front_token_threshold, back_token_threshold=0):
    input = []
    for index, row in df.iterrows():
        # select text according to the token threshold -> first FRONT
        # select first n (= token_theshold) tokens using the model tokenizer
        tokens = row[token_col_name][0:front_token_threshold]

        # combine tokens into txt
        tokens_txt = ''.join(tokens)

        # \n is converted by tokenizer to <0x0A>, we reverse this to get original length
        len_char = len(tokens_txt.replace("<0x0A>", "\n")) # get character length

        # select the same amount of characters as the tokens
        front_txt = row[text_col][0:len_char]

        # Check if back of document also given as input
        if back_token_threshold != 0:
            # select LAST n (= token_theshold) tokens using the model tokenizer
            tokens = row[token_col_name][-back_token_threshold:]

            # combine tokens into txt
            tokens_txt = ''.join(tokens)

            # \n is converted by tokenizer to <0x0A>, we reverse this to get original length
            len_char = len(tokens_txt.replace("<0x0A>", "\n")) # get character length

            # select the same amount of characters as the tokens
            back_txt = row[text_col][-len_char:]

            # combine front and back text
            input_txt = front_txt + ' ' + back_txt

        else:
            input_txt = front_txt

        input.append(input_txt)

    df['trunc_txt'] = input
    df['trunc_col'] = f"Truncation{token_col_name}Front{front_token_threshold}Back{back_token_threshold}"
    return df




# trunc = add_truncation_column(tok, 'text', 'GEITjeTokens', 50,50)
# display(trunc)

In [10]:
import itertools
front_thresholds = [0,100, 200, 500, 1000, 2000]
back_thresholds = [0,100,200,500,1000,2000]
all_combinations = list(itertools.product(front_thresholds, back_thresholds))

# remove combinations which have more than 2000 tokens.
all_combinations = [comb for comb in all_combinations if sum(comb) <= 2000 and sum(comb)>0]

# load file with baseline function
import sys
sys.path.append('../scripts/') 
import baseline as bf


from sklearn.svm import LinearSVC
# variables for text truncation
DATAFRAME = tok
TEXT_COL = 'text'
TOKENS_COL = 'LlamaTokens'

# variables for baseline
BASELINE_FUNCTION = LinearSVC()
MODEL_NAME = 'LinearSVC'
TRAIN_SET = 'train' # must be dev or train
TEST_SET = 'test' # must be val or test
SPLIT_COLUMN = '4split' #must be either 2split or 4split. 2split = data split into train and test. 4split = data split into train, test, dev and val. 
LABEL_COLUMN = 'label'
PREDICTION_PATH = f"{cf.output_path}/predictions/4baselineTruncationPredictions.pkl"
OVERVIEW_PATH = f"{cf.output_path}/overview/4baselineTruncationOverview.pkl"
TRUNC_COLUMN = 'trunc_col'
threshold_combinations =[(100,0), (200,0), (500,0), (1000,0), (2000,0), (100,100),(200,200), (500,500), (1000,1000), (0,100), (0,200), (0,500), (0,1000), (0,2000)]
# threshold_combinations = threshold_combinations[0:1]


In [11]:
for thresholds in threshold_combinations:
    front_threshold = thresholds[0]
    back_threshold = thresholds[1]
    trunc = add_truncation_column(DATAFRAME, TEXT_COL, TOKENS_COL, front_threshold,back_threshold)
    # display(trunc)
    # p_path = f"{front_threshold}{PREDICTION_PATH}"
    # o_path = f"{front_threshold}{OVERVIEW_PATH}"
    bf.run_baseline(BASELINE_FUNCTION,MODEL_NAME, trunc, SPLIT_COLUMN, TRAIN_SET, TEST_SET, TRUNC_COLUMN, LABEL_COLUMN, PREDICTION_PATH, OVERVIEW_PATH)

/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

                      precision    recall  f1-score   support

         Actualiteit       0.00      0.00      0.00       183
              Agenda       0.00      0.00      0.00       728
             Besluit       0.00      0.00      0.00       145
               Brief       0.00      0.00      0.00       396
          Factsheets       0.00      0.00      0.00        47
               Motie       0.31      1.00      0.47      1644
   Onderzoeksrapport       0.00      0.00      0.00       263
          Raadsadres       0.00      0.00      0.00       385
        Raadsnotulen       0.00      0.00      0.00        55
Schriftelijke Vragen       0.00      0.00      0.00       591
       Termijnagenda       0.00      0.00      0.00       207
          Voordracht       0.00      0.00      0.00       696

            accuracy                           0.31      5340
           macro avg       0.03      0.08      0.04      5340
        weighted avg       0.09      0.31      0.14      5340



/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/envs/azureml_py38_PT_TF/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/anaconda/e

                      precision    recall  f1-score   support

         Actualiteit       0.00      0.00      0.00       183
              Agenda       0.00      0.00      0.00       728
             Besluit       0.00      0.00      0.00       145
               Brief       0.00      0.00      0.00       396
          Factsheets       0.00      0.00      0.00        47
               Motie       0.31      1.00      0.47      1644
   Onderzoeksrapport       0.00      0.00      0.00       263
          Raadsadres       0.00      0.00      0.00       385
        Raadsnotulen       0.00      0.00      0.00        55
Schriftelijke Vragen       0.00      0.00      0.00       591
       Termijnagenda       0.00      0.00      0.00       207
          Voordracht       0.00      0.00      0.00       696

            accuracy                           0.31      5340
           macro avg       0.03      0.08      0.04      5340
        weighted avg       0.09      0.31      0.14      5340



In [ ]:
import pandas as pd
yeet = pd.read_pickle(PREDICTION_PATH)
# yeet = yeet.sort_values(by=['accuracy','macro_avg_f1'], ascending=False)
display(yeet)

,path,id,num_pages,MistralTokens,count_MistralTokens,LlamaTokens,count_LlamaTokens,trunc_txt,trunc_col,label,prediction,model,date
22200,/home/azureuser/cloudfiles/code/blobfuse/raads...,6734,1.0,"[▁X, %, ▁Geme, ente, <0x0A>, G, eme, ent, era,...",367,"[▁X, %, ▁Geme, ente, <0x0A>, G, eme, ent, era,...",361,X% Gemeente\nGemeenteraad RAAD\n% Amsterdam\nA...,TruncationLlamaTokensFront100Back0,Motie,Motie,LinearSVC,2024-04-23 11:09:41.278231+02:00
24121,/home/azureuser/cloudfiles/code/blobfuse/raads...,29365,3.0,"[▁V, N, 2, 0, 2, 3, -, 0, 1, 2, 8, 0, 6, ▁Ra, ...",2059,"[▁V, N, 2, 0, 2, 3, -, 0, 1, 2, 8, 0, 6, ▁Ra, ...",2000,"VN2023-012806 Raadscommissie voor Woningbouw, ...",TruncationLlamaTokensFront100Back0,Voordracht,Motie,LinearSVC,2024-04-23 11:09:41.278231+02:00
11816,/home/azureuser/cloudfiles/code/blobfuse/raads...,26699,2.0,"[▁V, N, 2, 0, 2, 1, -, 0, 2, 2, 2, 1, 6, ▁N, %...",740,"[▁V, N, 2, 0, 2, 1, -, 0, 2, 2, 2, 1, 6, ▁N, %...",731,VN2021-022216 N% Gemeente Raadscommissie voor ...,TruncationLlamaTokensFront100Back0,Voordracht,Motie,LinearSVC,2024-04-23 11:09:41.278231+02:00
6503,/home/azureuser/cloudfiles/code/blobfuse/raads...,15387,12.0,"[▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Geme, en...",9590,"[▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Geme, en...",9315,Gemeente Amsterdam\n% Gemeenteraad R\n% Defini...,TruncationLlamaTokensFront100Back0,Agenda,Motie,LinearSVC,2024-04-23 11:09:41.278231+02:00
22713,/home/azureuser/cloudfiles/code/blobfuse/raads...,23700,1.0,"[▁==, ▁e, @, <0x0A>, =, ▁Z, ò, <0x0A>, D, =, ▁...",1720,"[▁==, ▁e, @, <0x0A>, =, ▁Z, ò, <0x0A>, D, =, ▁...",1733,== e@\n= Zò\nD= ar\nrr LEER\n8 BOERE ZS BE\n. ...,TruncationLlamaTokensFront100Back0,Raadsadres,Motie,LinearSVC,2024-04-23 11:09:41.278231+02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11458,/home/azureuser/cloudfiles/code/blobfuse/raads...,1252,1.0,"[▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Geme, en...",598,"[▁Geme, ente, ▁Amsterdam, <0x0A>, %, ▁Geme, en...",592,Gemeente Amsterdam\n% Gemeenteraad R\n% Gemeen...,TruncationLlamaTokensFront100Back0,Motie,Motie,LinearSVC,2024-04-23 11:09:41.278231+02:00
24997,/home/azureuser/cloudfiles/code/blobfuse/raads...,21676,8.0,"[▁>, ▁<, ▁Geme, ente, ▁Ra, ads, in, format, ie...",6917,"[▁>, ▁<, ▁Geme, ente, ▁Ra, ad, sin, format, ie...",6729,> < Gemeente Raadsinformatiebrief\nAmsterdam\n...,TruncationLlamaTokensFront100Back0,Brief,Motie,LinearSVC,2024-04-23 11:09:41.278231+02:00
6148,/home/azureuser/cloudfiles/code/blobfuse/raads...,5549,1.0,"[▁X, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",605,"[▁X, ▁Geme, ente, ▁Amsterdam, ▁R, <0x0A>, G, e...",590,X Gemeente Amsterdam R\nGemeenteraad\n\nx Geme...,TruncationLlamaTokensFront100Back0,Motie,Motie,LinearSVC,2024-04-23 11:09:41.278231+02:00
18158,/home/azureuser/cloudfiles/code/blobfuse/raads...,4762,2.0,"[▁Gem, ee, ▁n, te, ▁Be, zo, ek, ad, res, <0x0A...",1482,"[▁Geme, e, ▁n, te, ▁Be, zo, ek, ad, res, <0x0A...",1462,Gemee nte Bezoekadres\nWeesperstraat 430\nAmst...,TruncationLlamaTokensFront100Back0,Motie,Motie,LinearSVC,2024-04-23 11:09:41.278231+02:00


In [ ]:
print(len(yeet))

19
